# Reduce size of geojson files

To reduce the file size and so have faster-loading maps, this notebook does two things to the features in a geojson file:

+ Reduce the precision of the coordinates (from typically 15 decimal places, e.g. [-0.099722888517677, 51.5167693121822]).
+ Remove any field except coordinates and the data for plotting, in this case: LSOA11NM.

## Setup

In [12]:
import geopandas
import shapely

## Import the geojson

In [2]:
path_to_file = '../data_geojson/ons_data/MSOA_Dec_2011_Boundaries_Super_Generalised_Clipped_BSC_EW_V3_2022_7707677027087735278.geojson'
geojson_ew = geopandas.read_file(path_to_file)

In [3]:
geojson_ew.head(3)

,OBJECTID,MSOA11CD,MSOA11NM,MSOA11NMW,BNG_E,BNG_N,LONG,LAT,GlobalID,geometry
0,1,E02000001,City of London 001,City of London 001,532378,181354,-0.093570,51.51560,a9f03568-7a0a-42b8-a23e-1271f76431e1,"POLYGON ((532946.065 181894.827, 533410.692 18..."
1,2,E02000002,Barking and Dagenham 001,Barking and Dagenham 001,548267,189693,0.138759,51.58659,f0ca54f0-1a1e-4c72-8fcb-85e21be6de79,"POLYGON ((549000.726 190873.464, 549120.966 18..."
2,3,E02000003,Barking and Dagenham 002,Barking and Dagenham 002,548259,188522,0.138150,51.57607,3772a2ec-b052-4000-b62b-2c85ac401a7f,"POLYGON ((548954.517 189063.241, 549085.611 18..."


In [4]:
geojson_ew.crs

<Derived Projected CRS: EPSG:27700>
Name: OSGB36 / British National Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: United Kingdom (UK) - offshore to boundary of UKCS within 49°45'N to 61°N and 9°W to 2°E; onshore Great Britain (England, Wales and Scotland). Isle of Man onshore.
- bounds: (-9.0, 49.75, 2.01, 61.01)
Coordinate Operation:
- name: British National Grid
- method: Transverse Mercator
Datum: Ordnance Survey of Great Britain 1936
- Ellipsoid: Airy 1830
- Prime Meridian: Greenwich

In [5]:
geojson_ew = geojson_ew.to_crs('CRS84')

In [6]:
geojson_ew.geometry

0       POLYGON ((-0.08519 51.52034, -0.07845 51.52151...
1       POLYGON ((0.14984 51.59701, 0.15111 51.58708, ...
2       POLYGON ((0.14841 51.58075, 0.14978 51.56970, ...
3       POLYGON ((0.19021 51.55268, 0.18602 51.54754, ...
4       POLYGON ((0.15441 51.56607, 0.14790 51.56109, ...
                              ...                        
7196    POLYGON ((-3.47955 53.31075, -3.47626 53.31516...
7197    POLYGON ((-2.96390 53.13277, -2.94691 53.12727...
7198    POLYGON ((-3.73326 52.50659, -3.73029 52.49864...
7199    MULTIPOLYGON (((-3.19550 51.45655, -3.19517 51...
7200    MULTIPOLYGON (((-3.17425 51.46478, -3.17559 51...
Name: geometry, Length: 7201, dtype: geometry

In [7]:
geojson_ew.crs

<Geographic 2D CRS: OGC:CRS84>
Name: WGS 84 (CRS84)
Axis Info [ellipsoidal]:
- Lon[east]: Geodetic longitude (degree)
- Lat[north]: Geodetic latitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
geojson_ew = geojson_ew[['MSOA11CD', 'MSOA11NM', 'geometry']]

In [9]:
# Reduce precision
geojson_ew.geometry = shapely.set_precision(geojson_ew.geometry, grid_size=0.001)

In [10]:
geojson_ew.head(3)

,MSOA11CD,MSOA11NM,geometry
0,E02000001,City of London 001,"POLYGON ((-0.07800 51.52200, -0.07400 51.51400..."
1,E02000002,Barking and Dagenham 001,"POLYGON ((0.15100 51.58700, 0.14200 51.58600, ..."
2,E02000003,Barking and Dagenham 002,"POLYGON ((0.15000 51.57000, 0.14700 51.56900, ..."


Write new geojson to file:

In [15]:
# Save as a .geojson:
save_name = 'MSOA_V3_reduced.geojson'
geojson_ew.to_file(save_name)